## Prueba Grafos

In [79]:
from DISClib.ADT import graph as g
from DISClib.ADT import list as lt
from DISClib.DataStructures import edge as e
from DISClib.ADT import stack
from DISClib.Algorithms.Graphs import bellmanford as bf
from DISClib.Algorithms.Graphs import bfs
from DISClib.Algorithms.Graphs import cycles as c

class grafo:
    def __init__(self, type='Undirected'):               # done
        if type == 'Directed':
            self.estructura = g.newGraph(size=10, directed=True)
        else:
            self.estructura = g.newGraph(size=10)

    def addNode_byValue(self, infoNodo):    #done
        infoNodo = str(infoNodo)
        self.estructura = g.insertVertex(self.estructura, infoNodo)

    def addEdge_byValue(self, infoNodo_1, infoNodo_2, weight=0): #done
        infoNodo_1 = str(infoNodo_1)
        infoNodo_2 = str(infoNodo_2)
        self.estructura = g.addEdge(self.estructura, infoNodo_1, infoNodo_2, weight)

    def deleteNode_byValue(self, infoNodo): #done
        infoNodo = str(infoNodo)
        self.estructura = g.removeVertex(self.estructura, infoNodo)

    def deleteEdge_byValue(self, infoNodo_1, infoNodo_2):
        #TODO
        pass

    def getEdgeValues(self):                #done
        lst = list()
        iter = lt.iterator(g.edges(self.estructura))
        aux = ()
        for i in iter:
            aux = (i['vertexA'],i['vertexB'],i['weight'])
            lst.append(aux)
        return lst            
    
    def getNodeValues(self):                #done
        lst = list()
        iter = lt.iterator(g.vertices(self.estructura))
        for i in iter:
            lst.append(i)
        return lst

    def isNodeValue(self, infoNodo):        #done
        infoNodo = str(infoNodo)
        return g.containsVertex(self.estructura, infoNodo)

    def findAdjacentNode(self, infoNodo):   #done
        infoNodo = str(infoNodo)
        lst = list()
        try:
            iter = lt.iterator(g.adjacents(self.estructura, infoNodo))
            for i in iter:
                lst.append(i)
        except:
            pass
        return lst
    
    def algorithms(self, algoritmo, infoNodo=None):
        table = list()
        if algoritmo == 'BellmanFord':
            search = bf.BellmanFord(self.estructura, infoNodo)
            for i in self.getNodeValues():
                costo = bf.distTo(search, i)
                path = list()
                if bf.hasPathTo(search, i):
                    iter = lt.iterator(bf.pathTo(search, i))
                    for j in iter:
                        path.append((j['vertexA'], j['vertexB']))
                table.append((i, costo, path))
                
        if algoritmo == 'BreadhtFisrtSearch':
            search = bfs.BreadhtFisrtSearch(self.estructura, infoNodo)
            for i in self.getNodeValues():
                path = list()
                if bfs.hasPathTo(search, i):
                    iter = lt.iterator(bfs.pathTo(search, i))
                    for j in iter:
                        path.append(j)
                table.append((i, None, path))
                
        if algoritmo == "DirectedCycle":
            search = c.DirectedCycle(self.estructura)
            if c.hasCycle(search):
                path = list()
                pathRta = c.cycle(search)
                while not stack.isEmpty(pathRta):
                    edge = stack.pop(pathRta)
                    path.append((edge['vertexA'], edge['vertexB'],edge['weight']))
                table.append(path)
        return table

In [80]:
import random
def create_n_randomEdges(nodos):
    edges = list()
    for i in range(len(nodos)):
        a = random.choice(nodos)
        b = random.choice(nodos)
        while a == b:
            b = random.choice(nodos)
        if (a,b) not in edges:
            num = random.random() + random.randint(1,50)
            edges.append((a,b,num))
    return edges

def create_n_random(n, string=False):
    nodos = list()
    for i in range(n*n):
        num = random.randint(1,50)
        if num not in nodos:
            if string:
                nodos.append(str(num))
            else:  
                nodos.append(num)
        if len(nodos) == n:
            break
    return nodos 

estructura = grafo()
long = random.randint(10,15)
nodos = create_n_random(long,string=True)
edges = create_n_randomEdges(nodos)
for i in nodos:
    try:
        estructura.addNode_byValue(str(i))
    except:
        e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
        raise Exception(e)
for i,j,k in edges:
    try:
        estructura.addEdge_byValue(i,j,k)
    except:
        e = '\tProblema al añadir el arco "('+i+ '->'+ j + ')", método addEdge_byValue()'
        raise Exception(e)

In [81]:
estructura.getNodeValues()

['44', '49', '1', '23', '16', '36', '4', '47', '32', '27', '35', '9', '37']

In [83]:
rta = estructura.algorithms('BellmanFord','44')
for i in rta:
    print(i)

('44', 0.0, [])
('49', inf, [])
('1', 22.244826683011475, [('44', '1')])
('23', 27.260489333147053, [('27', '23'), ('44', '27')])
('16', 31.961879400499356, [('27', '16'), ('44', '27')])
('36', 15.30863591183796, [('44', '36')])
('4', 38.551907121999, [('27', '4'), ('44', '27')])
('47', 33.67831993159018, [('23', '47'), ('27', '23'), ('44', '27')])
('32', 35.433464474915795, [('16', '32'), ('27', '16'), ('44', '27')])
('27', 24.731712204928204, [('44', '27')])
('35', 72.83384410841023, [('1', '35'), ('44', '1')])
('9', inf, [])
('37', 30.35198375090646, [('44', '37')])


In [ ]:
rta = estructura.algorithms('BreadhtFisrtSearch','10')
for i in rta:
    print(i)

('23', None, ['23', '30', '10'])
('30', None, ['30', '10'])
('10', None, ['10'])
('17', None, ['17', '30', '10'])
('38', None, [])
('40', None, ['40', '4', '23', '30', '10'])
('2', None, ['2', '23', '30', '10'])
('43', None, [])
('18', None, ['18', '30', '10'])
('32', None, ['32', '17', '30', '10'])
('4', None, ['4', '23', '30', '10'])
('14', None, [])


In [ ]:
rta = estructura.algorithms('DirectedCycle')
print(rta)

[[('23', '32', 29.36205158489865), ('32', '23', 29.36205158489865)]]
